# PIRIS + Bayesian Optimization
**Bayesian Optimization for hyperparameter tunning in PIRIS**


# Librerias

In [ ]:
#Importamos ls librerias necesarias
import numpy as np
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import plotly.io as pio
from sklearn.cluster import kmeans_plusplus
import pandas as pd
import tensorflow as tf
import optuna
from optuna.samplers import GPSampler
from optuna.visualization import (
    plot_param_importances,
    plot_contour
)

# kmeans++

**Para el k-means++, necesitamos unos limites alrededor de la superficie**

In [ ]:
def calcular_limites_superficie(posiciones, geometria, padding=2.0):
    """
    Calcula los límites en (x,y) de una estructura atómica plana, añadiendo un margen.
    O el centro si es una estrutura no plana.

    Args:
        posiciones (np.ndarray): Un array de forma (N, 3) con las coordenadas (x,y,z)
                                 de los átomos de la superficie.
        padding (float): Un margen en Angstroms que se añadirá en cada dirección
                         (min y max) a los límites de la estructura.

    Returns:
        Un diccionario conteniendo la informacion para la selección de candidatos. 
               Formato: {
                   tipo: (plano o 3D),
                   limite: tupla de limites en xy ((x_min,x_max),(y_min,y_max)) o centro de la muestra (x_center,y_center,z_center)
               }
    """
    if posiciones.ndim != 2 or posiciones.shape[1] < 2:
        raise ValueError("El array de posiciones debe tener forma (N, 2) o (N, 3).")
        
    if geometria=="planar":
        # Extraer las coordenadas x e y
        x_coords = posiciones[:, 0]
        y_coords = posiciones[:, 1]
        
        # Encontrar los valores mínimos y máximos
        x_min, x_max = np.min(x_coords), np.max(x_coords)
        y_min, y_max = np.min(y_coords), np.max(y_coords)
        
        # Aplicar el padding
        limites_x = (x_min - padding, x_max + padding)
        limites_y = (y_min - padding, y_max + padding)
        
        print(f"Límites de la estructura calculados:")
        print(f"  - Rango X (con padding): ({limites_x[0]:.2f}, {limites_x[1]:.2f})")
        print(f"  - Rango Y (con padding): ({limites_y[0]:.2f}, {limites_y[1]:.2f})")
        info_k_means = {
            "tipo": "planar",
            "limite": (limites_x, limites_y)
        }
        return info_k_means
        
    elif geometria=="3D":
        centro = np.mean(posiciones, axis=0)
        info_k_means = {
            "tipo":"3D",
            "limite": centro
        }
        return info_k_means
    else:
        raise ValueError(f"Tipo de geometría '{geometria}' no soportado.")

**Con los limites ya calculados, generamos las posibles opciones de posicion inicial**

In [ ]:
def starting_point_suggestions(info_k_means, puntos_por_eje):
    """
    Generalos candidatos para que k-means++ elija posiciones iniciales 
    Args:
        geometria: informacion para la eleccion de candidatos (dict)
        puntos_por_eje (int): Número de puntos a generar por cada eje.

    Returns:
        para geometria planar: malla de posibles puntos (x,y)
        para geometria 3D: vectores de direccion en una esfera unitaria
    """

    #Extraemos el tipo de geometria
    tipo = info_k_means["tipo"]
    if tipo == "planar":
        limites_x, limites_y = info_k_means["limite"]
        # Generar los puntos en cada eje usando los límites calculados
        x_coords = np.linspace(limites_x[0], limites_x[1], puntos_por_eje)
        y_coords = np.linspace(limites_y[0], limites_y[1], puntos_por_eje)
        
        # Crear la malla usando meshgrid
        xx, yy = np.meshgrid(x_coords, y_coords)
        
        # Aplanar y combinar en un array de coordenadas (N, 2)
        malla = np.vstack([xx.ravel(), yy.ravel()]).T
        
        print(f"Malla de {malla.shape[0]} posiciones candidatas creada sobre los límites definidos.")
        return malla
    elif tipo == "3D":
        # Generar vectores aleatorios y normalizarlos para obtener puntos en una esfera unitaria
        vecs = np.random.randn(num_candidatos, 3)
        vecs /= np.linalg.norm(vecs, axis=1, keepdims=True)
        return vecs # Estos son nuestros vectores de dirección candidatos
    else:
        raise ValueError("Forma de configuración no reconocida.")

**Teniendo las opciones, solo falta elegir**

In [ ]:
def generate_xy(options, k_simulaciones, num_ni_por_simulacion, random_seed=42):
    """
    Selecciona 'k_simulaciones' conjuntos de 'num_ni_por_simulacion' posiciones iniciales
    cada uno.

    Args:
        options(np.ndarray): Array de posiciones candidatas (N, 2) si es geometria planar, o  vectores unitarios(N,3) si es
        un estructura 3D.
        k_simulaciones (int): Número de simulaciones/configuraciones a generar.
        num_ni_por_simulacion (int): Número de iones en cada simulación.
        random_seed (int): Semilla para reproducibilidad.

    Returns:
        list: Una lista de 'k_simulaciones' arrays. Cada array tiene forma 
              (num_ni_por_simulacion, 2) si es geometria plana, o (num_ni_por_simulacion,3) si es 
              3D, y representa una configuración inicial.
    """
    
    total_puntos = k_simulaciones * num_ni_por_simulacion
    puntos_seleccionados, _ = k_means_plusplus(options, n_clusters=total_puntos, random_state=random_seed)
    configuracion = np.array_split(puntos_seleccionados, k_simulaciones)
    return configuracion


# Creación de los iones que se van a adsorber

**La optimizacion de hiperparametros contemplara la distancia de los iones a la muestra como un hiperparametro** 

In [ ]:
def build_ions(configuracion, opt_param, info_k_means):
    """Recibe posiciones iniciales pre definidas y la distancia para combinarlas y generar los iones
    a adsorber
    args: 
        configuracion:array con un conjunto de posiciones iniciales preseleccionadas.
        opt_param: float de distancia optimizable por optuna
    returns:
        np.ndarray: Array de forma (num_ni, 3) con las posiciones completas (x,y,z) de los iones"""
    
    if info_k_means["tipo"] == "planar":
        #Tomamos la cantidad de iones del array de coordenaads xy
        num_ni = configuracion.shape[0]
        # Crea una columna de 'z' con el mismo valor para todos los iones
        z_column = np.full((num_ni, 1), opt_param)
        
        # Concatena las posiciones (x,y) con la columna z
        ion_positions = np.hstack([xy_positions, z_column])
        
        return ion_positions
    elif info_k_means["tipo"] == "3D":
        centro = info_k_means["limite"]
        distancia = opt_param
        return centro + distancia * configuracion
    
    else:
        raise ValueError("Forma de configuración no reconocida.")

# Definición del costo para entrenamiento y evaluación.

**Costo de entrenamiento**

In [ ]:
# Lennard-Jones potential function
@tf.function
def physical_potential(r, epsilon, sigma, q_j):
    sr6 = tf.math.pow(sigma / r, 6)
    sr12 = tf.math.pow(sr6, 2)
    lj = 4.0 * epsilon * (sr12 - sr6) + epsilon # Sumamos epsilon para dejar el fondo del pozo en 0
    coulomb = k_e * q_Ni * q_j / tf.maximum(r, 1e-10)
    return lj + coulomb

# Funcion para calcular el potencial de informacion, individual y cruzado
@tf.function
def calculate_cross_information_potential(points1, points2, sigma):
    """
    Calcula el Potencial de Información Cruzado (V) entre dos conjuntos de puntos.
    Si points1 y points2 son el mismo, calcula el Potencial de Información (IP).
    """
    diff = tf.expand_dims(points1, 1) - tf.expand_dims(points2, 0)
    sq_dists = tf.reduce_sum(tf.square(diff), axis=-1)
    kernel_matrix = tf.exp(-sq_dists / (2.0 * sigma**2))
    return tf.reduce_mean(kernel_matrix)
    
# Función de costo para el entrenamiento del modelo
@tf.function
def get_training_loss(positions, elements, ni_positions, pri_weight, lambda_pri, sigma_pri):
     """
    Calcula la energía total de las interacciones entre los iones (ni_positions) y la superficie (positions),
    utilizando un potencial fisico y una regularizacion por PRI.
    Parameters:
        Ni_positions: tensor con posiciones de los iones (num_iones, 3)
        Positions: tensor con posiciones de los átomos de la superficie (num_atoms, 3)
        Elements: tensor con etiquetas de tipo de atomo (num_superficie,)
        pri_weight,lambda_pri,sgma_pri: hiperparametros escalares que modelan el peso de la regulaización
    Returns:
        Energía total del sistema (escalar)
        Energía total de cada ion (num_iones,)
    """
    
    #----------------------INTERACCIONES ION-NANOESTRUCTURA-------
    # Expandir dimensiones para obtener todas las combinaciones de distancias ion-atomo
     pos_exp = tf.expand_dims(positions, axis=1)  # (num_atoms, 1, 3)
     ni_exp = tf.expand_dims(ni_positions, axis=0)  # (1, num_iones, 3)
   
     r = tf.norm(pos_exp - ni_exp, axis=2)  # Uso la norma para calcular la magnitud de la distancia 
                                # entre cada ion y cada átomo de la superficie (num_iones, num_atoms)
     
    # Casteamos las etiquetas para que tengan la forma de r
     elements_exp = tf.reshape(elements, (-1, 1))  # (num_atoms, 1)

    # Creamos una mascara para etiquetar las interacciones
     is_O = tf.equal(elements_exp, "O")
     is_Zn = tf.equal(elements_exp, "Zn")


    # Calculamos la energía aplicando cortes de distancia de interacción
     energy_O = tf.where(r < r_cutoffO_Ni,
                        physical_potential(r, EpsilonO_Ni, SigmaO_Ni,q_O),
                        tf.zeros_like(r))
     energy_Zn = tf.where(r < r_cutoffZn_Ni,
                         physical_potential(r, EpsilonZn_Ni, SigmaZn_Ni, q_Zn),
                         tf.zeros_like(r))
    # Filtramos el valor de la energía dependiendo de la etiqueta
     atoms_energy_matrix = tf.where(is_O, energy_O,
                            tf.where(is_Zn, energy_Zn, tf.zeros_like(r)))
    # Energía total por ion (sumando sobre átomos)
     energy_per_ion = tf.reduce_sum(atoms_energy_matrix, axis=0)  # shape: (num_ions,)

    # --------
    # Esta sección maneja la energía promedio por ion, por ahora no se esta usando
    # Calculamos la energia promedio por ion considerando que no todos interactuan con la misma
    # cantidad de atomos.
    # Definimos mascaras para contar las interacciones validas
     mask_O = tf.cast(tf.less(r, r_cutoffO_Ni), tf.float32) * tf.cast(is_O, tf.float32)
     mask_Zn = tf.cast(tf.less(r, r_cutoffZn_Ni), tf.float32) * tf.cast(is_Zn, tf.float32)
     interaction_mask_atoms = tf.maximum(mask_O, mask_Zn)  # shape: (num_atoms, num_ions)
    # Número de interacciones válidas por ion
     count_per_ion = tf.reduce_sum(interaction_mask_atoms, axis=0)  # shape: (num_ions,)
     count_per_ion_safe = tf.where(count_per_ion == 0, 1.0, count_per_ion)  # evitar división por cero
    # Energia promedio de cada ion
     mean_energy_per_ion = energy_per_ion / count_per_ion_safe  # shape: (num_ions,)
    #---------


    #----------------------INTERACCIONES ION-ION-------
    # Calcula matriz de vectores diferencia entre iones
     diff_ion = tf.expand_dims(ni_positions, 0) - tf.expand_dims(ni_positions, 1)  # (num_ions, num_ions, 3)
    # Calcula matriz de distancias usando norma
     r_ion = tf.norm(diff_ion, axis=-1)  # (num_ions, num_ions)

    # Construye máscaras para excluir la diagonal y aplicar cutoff
     eye = tf.eye(tf.shape(ni_positions)[0], dtype=tf.bool)  # diagonal True
     mask_offdiag = tf.logical_not(eye)  # off-diagonal True
     mask_cutoff = tf.less(r_ion, r_cutoffNi_Ni)  # True si r < cutoff
     interaction_mask_ions = tf.logical_and(mask_offdiag, mask_cutoff)  # pares válidos distintos
    # Prepara distancias válidas evitando divisiones por cero
     valid_r = tf.where(interaction_mask_ions, tf.maximum(r_ion, 1e-10), tf.ones_like(r_ion))

    # Calcula energía Lennard-Jones solo en pares válidos
     ion_energy_matrix = tf.where(interaction_mask_ions,
                             physical_potential(valid_r, EpsilonNi_Ni, SigmaNi_Ni, q_Ni),
                             tf.zeros_like(r_ion))
    # Energías totales ion_ion
     ion_ion_energy_per_ion = tf.reduce_sum(ion_energy_matrix, axis=0)  # (num_ions,)
    
    # --------
    # Esta sección maneja la energía promedio por ion, por ahora no se esta usando
    # Número de interacciones válidas por ion
     count_per_ion_ion = tf.reduce_sum(tf.cast(interaction_mask_ions, tf.float32), axis=0)  # (num_ions,)
     count_per_ion_ion_safe = tf.where(count_per_ion_ion == 0, 1.0, count_per_ion_ion)
    # Energias promedio ion_ion
     mean_ion_ion_energy_per_ion = ion_ion_energy_per_ion / count_per_ion_ion_safe  # (num_ions,)
    #---------------
    
    # calcular la energía total del sistema y la energía por ion
     ion_ion_total_energy = tf.reduce_sum(ion_ion_energy_per_ion) * 0.5 #Multiplico por 0.5 porque la energia aparece una vez para cada ion
     energy_total_per_ion = energy_per_ion + ion_ion_energy_per_ion
     energy_total = tf.reduce_sum(energy_per_ion)+ion_ion_total_energy

    #--------------------REGULARIZACIÓN CON PRI----------------------
    # Analogía: positions -> majority class | ion_positions -> undersampled majority class 

     epsilon_log = 1e-10  # Para estabilidad numérica en el logaritmo

    # Calcular potenciales de información
     ip_ions = calculate_cross_information_potential(ni_positions, ni_positions, sigma_pri)
     ip_surface = calculate_cross_information_potential(positions, positions, sigma_pri) # Es constante, pero se calcula aquí por simplicidad
     cip_ions_surface = calculate_cross_information_potential(ni_positions, positions, sigma_pri)
    # Calcular entropías cuadráticas (H2 = -log(V))
     H2_ions = -tf.math.log(ip_ions + epsilon_log)
     H2_surface = -tf.math.log(ip_surface + epsilon_log)
     H2_cross = -tf.math.log(cip_ions_surface + epsilon_log)

    # Calcular Divergencia de Cauchy-Schwarz (D_cs= 2H2(ion,surface)- H2(ion)- H2(surface)) - Eq. 4 del paper
     D_cs = 2 * H2_cross - H2_ions - H2_surface
    # Calcular costo PRI - Eq. 7 del paper
    # J(X_hat) = H2(X_hat) + lambda * D_cs(X_hat, X)
     pri_cost = (1- lambda_pri)*H2_ions + 2*lambda_pri * D_cs

    #------------COMBINAR Y RETORNAR----------------

     total_loss = energy_total + pri_weight * pri_cost
    
     return  total_loss, energy_total_per_ion

**Costo de evaluación**

In [ ]:
@tf.function
def calculate_final_system_energy(positions, elements, ni_positions):
    """
    Calcula la energía del sistema sin PRI y corrige el shift potencial de la energía
    """
    pos_exp = tf.expand_dims(positions, axis=1)
    ni_exp = tf.expand_dims(ni_positions, axis=0)
    r = tf.norm(pos_exp - ni_exp, axis=2)
    elements_exp = tf.reshape(elements, (-1, 1))
    is_O = tf.equal(elements_exp, "O")
    is_Zn = tf.equal(elements_exp, "Zn")
    
    energy_O = tf.where(r < r_cutoffO_Ni, physical_potential(r, EpsilonO_Ni, SigmaO_Ni, q_O), tf.zeros_like(r))
    energy_Zn = tf.where(r < r_cutoffZn_Ni, physical_potential(r, EpsilonZn_Ni, SigmaZn_Ni, q_Zn), tf.zeros_like(r))
    atoms_energy_matrix = tf.where(is_O, energy_O, tf.where(is_Zn, energy_Zn, tf.zeros_like(r)))
    energy_per_ion = tf.reduce_sum(atoms_energy_matrix, axis=0)

    diff_ion = tf.expand_dims(ni_positions, 0) - tf.expand_dims(ni_positions, 1)
    r_ion = tf.norm(diff_ion, axis=-1)
    eye = tf.eye(tf.shape(ni_positions)[0], dtype=tf.bool)
    mask_offdiag = tf.logical_not(eye)
    mask_cutoff = tf.less(r_ion, r_cutoffNi_Ni)
    interaction_mask_ions = tf.logical_and(mask_offdiag, mask_cutoff)
    valid_r = tf.where(interaction_mask_ions, tf.maximum(r_ion, 1e-10), tf.ones_like(r_ion))
    ion_energy_matrix = tf.where(interaction_mask_ions, physical_potential(valid_r, EpsilonNi_Ni, SigmaNi_Ni, q_Ni), tf.zeros_like(r_ion))
    ion_ion_energy_per_ion = tf.reduce_sum(ion_energy_matrix, axis=0)
    
    ion_ion_total_energy = tf.reduce_sum(ion_ion_energy_per_ion) * 0.5
    physical_energy_total = tf.reduce_sum(energy_per_ion) + ion_ion_total_energy
    
    # --- Apply the final epsilon correction ---
    mask_O = tf.cast(tf.less(r, r_cutoffO_Ni), tf.float32) * tf.cast(is_O, tf.float32)
    mask_Zn = tf.cast(tf.less(r, r_cutoffZn_Ni), tf.float32) * tf.cast(is_Zn, tf.float32)
    num_interactions_O = tf.reduce_sum(tf.cast(mask_O, tf.float32))
    num_interactions_Zn = tf.reduce_sum(tf.cast(mask_Zn, tf.float32))
    num_interactions_ionion = tf.reduce_sum(tf.cast(interaction_mask_ions, tf.float32))*0.5 #igual que en la energía, las interacciones se cuentan doble
    total_epsilon_shift = (
        num_interactions_O * EpsilonO_Ni +
        num_interactions_Zn * EpsilonZn_Ni +
        num_interactions_ionion * EpsilonNi_Ni
    )
    corrected_energy_total = physical_energy_total - total_epsilon_shift
    
    return corrected_energy_total, energy_per_ion + ion_ion_energy_per_ion

**Definmos un exponential decay learning rate**

In [ ]:
def exp_decay_lr(initial_lr, decay_rate, epoch):
    return initial_lr * decay_rate ** (epoch/5)

**Definimos clase para early stop**

In [ ]:
# Early_stop Callback class
class EarlyStoppingCallback:
    def __init__(self, patience, min_delta):
        self.patience = patience
        self.min_delta = min_delta
        self.wait = 0
        self.best_loss = float('inf')
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, loss):
        if loss < self.best_loss - self.min_delta:
            self.best_loss = loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                return True  # Signal to stop training
        return False  # Continue training

# Bayesian Optimization

**Definimos la funcion objetivo de la optimización bayesiana**

In [ ]:
def objective(trial):
    """
    Objective function for Optuna to minimize.
    It runs the PIRIS simulation for a given set of hyperparameters.
    """
    # --- 1. Suggest hyperparameters from the trial object ---
    # distance: range is based on physical constants
    distance_min_bound = float(0)
    distance_max_bound = float(6 * SigmaZn_Ni.numpy()) # A reasonable upper bound based on cutoff radius and PRI reach
    distance_initial = trial.suggest_float("distance", distance_min_bound, distance_max_bound)

    # PRI hyperparameters
    pri_weight_val = trial.suggest_float("pri_weight", 1.0, 10.0) #Up to 10 given the magnitude of H vs PRI loss
    lambda_pri_val = trial.suggest_float("lambda_pri", 0.0, 1.0)
    
    # Sigma_pri: search space follows the magnitude of z
    sigma_pri_val = trial.suggest_float("sigma_pri", 1.0, 8*SigmaZn_Ni.numpy(), log=True)# A reasonable range follows z distribution

    # Use the learning rate and decay rate suggested by the trial
    eta0 = trial.suggest_float("learning_rate", 1e-3, 5e-1, log=True)
    decay_rate = trial.suggest_float("decay_rate", 0.90, 0.99)

    # --- 2. Setup the simulation for this trial ---
    # Traemos de los atributos del trial la posicion inicial
    position_str = trial.study.user_attrs["starting_position"]
    # Convert the JSON string back into a Python list/array
    configuracion = np.array(json.loads(position_str))
    # Create the initial ion positions with the suggested opt_param
    initial_ni_positions = build_ions(configuracion, distance_initial, info_k_means)
    ion = tf.Variable(initial_ni_positions, dtype=tf.float32)

    # Cast hyperparameters to TensorFlow constants for this trial
    pri_weight = tf.constant(pri_weight_val, dtype=tf.float32)
    lambda_pri = tf.constant(lambda_pri_val, dtype=tf.float32)
    sigma_pri = tf.constant(sigma_pri_val, dtype=tf.float32)

    # --- 3. Run the training loop ---
    epochs = 200  # Reduced for faster optimization trials
    optimizer = tf.keras.optimizers.Adam(learning_rate=eta0)

    for epoch in range(epochs):
        eta = exp_decay_lr(eta0, decay_rate=decay_rate, epoch=epoch)
        optimizer.learning_rate.assign(eta)
        
        with tf.GradientTape() as g:
            g.watch(ion)
            H_loss, _ = get_training_loss(sample_atoms, sample_elements, ion.value(), pri_weight, lambda_pri, sigma_pri)
        
        grad_ = g.gradient(H_loss, ion)
        
        # Check for NaN gradients which can happen with extreme hyperparameters
        if tf.reduce_any(tf.math.is_nan(grad_)):
            # If gradients are NaN, the trial is bad. Return a large value.
            return 1e6

        optimizer.apply_gradients(zip([grad_], [ion]))

    # --- 4. Calculate final energy and return it ---
    final_energy, _ = calculate_final_system_energy(sample_atoms, sample_elements, ion.value())
    
    return final_energy.numpy()

**Optimization loop**

In [ ]:
def optimization(positions, puntos_por_eje, k, num_ni, num_trials, random_seed=42):
    """Funcion para ejecutar el loop de optimizacion
    args: 
        positions: posiciones que componen la superficie del adsorbente (np.array)
        puntos_por_eje: cantidad de puntos a tomar como muestra desde el kmeans++ en cada eje (int)
        k: número de muestras a elegir por kmeans++ (int)
        num_ni: número de iones a adsorber de forma simultanea (int)
        num_trials: número de pruebas para realizar por cada posicion inicial (int)
    Returns:
        Studies: lista con todos los k estudios realizados y sus parametros (list)
        Trials: lista con los mejores trials de cada estudio (list)"""

    #Definimos el box alrededor de la superficie para los candidatos a xy
    limit_x,limit_y = calcular_limites_superficie(positions, padding=2.0)
    
    #Definimos la malla de candidatos xy
    malla = xy_candidate_selection(limit_x, limit_y, puntos_por_eje)
    
    #Ejecutamos el algoritmo k-means++ para elegir los candidatos
    configuraciones=generate_xy(malla, k, num_ni, random_seed)
    
    Studies = [] #Estudios completos para el plot de importancia
    Trials = [] #Mejor resultado para cada estudio
    
    for i, xy_positions in enumerate(configuraciones):
        print("Starting position: ",i)
        print("x,y for try ",i,": ",xy_positions)
        study_name = f'Starting_position_{i}'
        # Creamos el objeto estudio, dando nombre, direccion (minimizar) y usando un GP como sampler.
        study = optuna.create_study(study_name=study_name, direction='minimize', sampler=GPSampler(seed=10))
        # Vamos a necesitar la posicion inicial despues, así que la guardamos como atributo del estudio
        list_positions = xy_positions.tolist()
        study.set_user_attr("starting_position", json.dumps(list_positions))
        study.set_user_attr("position_index", i) # guardamos el indice tambien por si se llegara a necesitar
        
        # Optimizamos, optuna llama la funcion objetivo n_trials veces
        study.optimize(objective, n_trials=num_trials)
        
        # --- guardamos resultados ---
        Studies.append(study) # el estudio completo
        
        trial = study.best_trial # tomamos el mejor trial
    
        Trials.append(trial) # y lo guardamos por separado
    return Studies, Trials

In [ ]:
def optimization_summary(Trials):
    """Toma los datos de la optimizacion y hace un dataframe con los mejores trials
    Args:
        Trials: Lista con los mejores trials por posicion inicial (list)
    Returns:
        data: Informacion de los hiperparametros y resultados de cada Trial (pd.dataframe)"""
    records =[]
    for trial in Trials:
        record={"Energy":trial.value}
        record.update(trial.params)
        records.append(record)
    data = pd.DataFrame(records)
    print("Summary of the best trials:")
    print(data)
    return data

**Visualizacion de los resultados de la optimización**

**Importance plot**

In [ ]:
def importance_plot(Studies):
    """Lee los datos de los estudios de optuna y grafica la importancia de cada hiperparametro con su respectiva varianza
    Args: 
        Studies: Lista con todos los datos de cada estudio realizado para cada posicion inicial (List)
    Returns:
        None"""
    # Creamos una lista con las importancias de todos los estudios
    all_importances = [optuna.importance.get_param_importances(s) for s in Studies]
    
    #Lo convertimos a dataframe al ser una lista de diccionarios
    df_importances = pd.DataFrame(all_importances)
    #por si algun parametro se queda fuera, cambiamos NaN a 0
    df_importances = df_importances.fillna(0)
    
    # Calculamos la media y la desviacion estandar de cada parametro a lo largo de las columnasdel df
    mean_importances = df_importances.mean().sort_values(ascending=False)
    std_importances = df_importances.std().loc[mean_importances.index] # aseguramos que mantenga el orden
    
    # Y ploteamos on plt
    plt.figure(figsize=(12, 8))
    bars = plt.barh(
        y = mean_importances.index,
        width = mean_importances,
        xerr=std_importances, # This adds the error bars (std dev)
        capsize=5, # Puts caps on the error bars
        color=sns.color_palette("viridis_r", len(mean_importances))
    )
    
    #Invierte el eje para que el y mas grande esté arriba
    plt.gca().invert_yaxis()
    
    # añadimos titulos y labels
    plt.title('Importancia promedio de los hiperparametros para todas las posiciones iniciales', fontsize=16, fontweight='bold')
    plt.ylabel('Hiperparametros', fontsize=12)
    plt.xlabel('Imporancia media', fontsize=12)
    plt.tight_layout() # Adjust layout to make room for labels
    
    
    # Show the parameter importances
    plt.show()
    # En lugar de show vale la pena guardar la imagen para pegarla en un overleaf

**Contour plots**

In [ ]:
def contour_plot(Trials,Studies):
    """Lee la informacion de la optimizacion para hacer los diagramas de contorno de los hiperparametros de interes
    Args:
        Trials: Lista con los mejores trials de cada posicion inicial (list)
        Studies: Inforamcion completa de todos los estudios para todas las posiciones iniciales (list)
    Retruns:
        best_study: Datos de la posicion inicial con el mejor rendimiento (optuna.study object)"""
    # Contour plot de los hiperparametros más importantes
    
    # Identifica el mejor trial de todos
    best_overall_trial = min(Trials, key=lambda t: t.value)
    
    # Determinar a que posicion inicial pertenece
    best_study = None
    for study in Studies:
        # We compare the best trial of the current study in the loop
        # with the 'best_overall_trial' we found earlier.
        # FrozenTrial objects can be compared directly for equality.
        if study.best_trial == best_overall_trial:
            best_study = study
            break # We found the matching study, so we can exit the loop
    
    # It's good practice to handle the case where we somehow don't find a match.
    if best_study is None:
        raise RuntimeError("Could not find the parent study for the best trial. This should not happen.")
    
    # crear y mostrar 2 plots para diferentes parametros
    z_sigma_plot = plot_contour(best_study, params =["z","sigma_pri"])
    pri_physics_plot = plot_contour(best_study, params =["pri_weight","lambda_pri"])
    z_sigma_plot.show()
    pri_physics_plot.show()
    #Igual que arriba, tenemos que guardar las imagenes en lugar de solo mostrarlas
    return best_study

**Llamamos toda la visualizacion de resultados en una sola función**

In [ ]:
def optimization_results(Studies, Trials):
    """Llama todas las funciones de visualizacion de resultados
    Args:
        Studies: Inforamcion completa de todos los estudios para todas las posiciones iniciales (list)
        Trials: Lista con los mejores trials de cada posicion inicial (list)
    Returns:
        best_study: Datos de la posicion inicial con el mejor rendimiento (optuna.study object)"""
    
    optimization_summary(Trials)
    importance_plot(Studies)
    best_study = contour_plot(Trials,Studies)
    return best_study

# Entrenamiento y resultados

**Loop de entrenamiento con los hiperparametros optimizados**

In [ ]:
def training_loop(best_study, epochs,  positions, sample_atoms, sample_elements):
    """Realiza todo el loop de entrenamiento cn los hiperparametros optimizados
    Args: 
        best_study: datos de la mejor posicion inicial (optuna.study object)
        epochs: número de epocas para el entrenamiento (int)
        positions: coordenadas de la superficie para las graficas (np.array)
        sample_atoms: coordenadas de la superficie para el entrenamiento (tf.tensor (float32))
        sample_elements: etiquetas del tipo de elemento para el entrenamiento (tf.tensor (str))
    Returns:
        ion_final: posicion inicial definitiva de (los) ion(es), optimizada por optuna (tf.variable (float32))
        ion_final_:posicion final de (los) ion(es), optimizada por el entrenamiento en formato numpy(np.float32)
        starting_pos: posicion inicial definitiva de (los) ion(es), optimizada por optuna en formato numpy (np.float32)
        loss_history: datos de la evolucion del loss durante el entrenamiento (list)"""
    
    # Get the best hyperparameters from the study
    best_params = best_study.best_trial.params
    print("\nBest Hyperparameters Found:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    
    # Extract the starting position from the study's user attributes ---
    best_starting_position_str = best_study.user_attrs["starting_position"]
    # Usamos json para cargar la posicion que guardamos antes
    best_starting_position_list = json.loads(best_starting_position_str)
    best_starting_position = np.array(best_starting_position_list) #Turn back to numpy array
    
    
    # Set up the final simulation using these parameters
    final_z = best_params['z'] #Altura optima inicial de los iones
    #Hiperparametros de regularización
    final_pri_weight = tf.constant(best_params['pri_weight'], dtype=tf.float32) #Peso PRI vs Fisica
    final_lambda_pri = tf.constant(best_params['lambda_pri'], dtype=tf.float32) #Peso Entropia ion-ion vs cruzada
    final_sigma_pri = tf.constant(best_params['sigma_pri'], dtype=tf.float32) # Ancho del kernel RBF
    #Hiperparametros del entrenamiento
    final_lr = best_params['learning_rate']
    final_decay = best_params['decay_rate']
    
    # Re-initialize the ion at the optimal starting height
    final_initial_positions = build_ions(best_starting_position, z=final_z)
    ion_final = tf.Variable(final_initial_positions, dtype=tf.float32)
    ion_final_ = ion_final.numpy()
    optimizer_final = tf.keras.optimizers.Adam(learning_rate=final_lr)
    
    #Pintamos la posicion inicial de los iones con la altura final
    fig = plt.figure(figsize=(6, 6))  # Adjust figure size if needed
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c='b', s=20, label='fcc_positions')
    ax.scatter(ion_final_[:, 0], ion_final_[:, 1], ion_final_[:, 2], c='r', s=40, marker='x', label='ion_')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    
    plt.show()
    
    # Loop de entrenamiento final
    
    starting_pos =ion_final_.copy()
    loss_history = []#Para guardar y graficar la energía del sistema
    
    early_stopping = EarlyStoppingCallback(patience=5, min_delta=0.00001)
    
    for epoch in range(epochs):
        print(f'Epoch {epoch+1}/{epochs}')
        eta = exp_decay_lr(final_lr, decay_rate=final_decay, epoch=epoch)
        optimizer_final.learning_rate.assign(eta)
        
        with tf.GradientTape() as g:
            g.watch(ion_final)
            H_loss,ion_total_energies = get_training_loss(sample_atoms,sample_elements,ion_final,
                                                          final_pri_weight,final_lambda_pri,final_sigma_pri)
            loss_history.append(H_loss)
        grad_ = g.gradient(H_loss, ion_final)
        optimizer_final.apply_gradients(zip([grad_], [ion_final]))
        print(f'Loss: {H_loss.numpy()} MeanGrad: {tf.math.reduce_mean(grad_,axis=0).numpy()}')
    
        # Call the callback's on_epoch_end method
        """if early_stopping.on_epoch_end(epoch, H_loss.numpy()):
            print(f'Early stopping at epoch {early_stopping.stopped_epoch + 1}')
            break"""
        if (epoch + 1) % 5 == 0:
            print(f'| Loss: {H_loss.numpy():.4f} | LR: {optimizer_final.learning_rate.numpy():.4f}')
        ion_final_ =  ion_final.numpy()
        gradN = -10*eta*grad_.numpy()
        # For quiver, we need to create a 3D representation of the gradient
        # Assuming gradN is a 2D array (num_ions, 2)
        gradN_3D = np.zeros(gradN.shape,dtype=gradN.dtype)  # Initialize with zeros for z-component
        gradN_3D = gradN  # Copy x and y components from gradN
        if (epoch)==epochs//2:
        
        
            fig = plt.figure(figsize=(6, 6))
            ax = fig.add_subplot(111, projection='3d')
        
            ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c='b', s=20, label='fcc_positions')
            ax.scatter(ion_final_[:, 0], ion_final_[:, 1], ion_final_[:, 2], c='r', s=40, marker='x', label='ion_')
        
            
        
            ax.quiver(ion_final_[:, 0], ion_final_[:, 1], ion_final_[:, 2], gradN_3D[:, 0], gradN_3D[:, 1], gradN_3D[:, 2],
                      color='g', length=0.7, normalize=True, label='Gradient Vectors')  # Adjust length and normalize as needed
        
            #ax.set_xlim([-1*Sigma_I_L,x_lim +1*Sigma_I_L])
            #ax.set_ylim([-1*Sigma_I_L,x_lim +1*Sigma_I_L])
            #ax.set_zlim([-1*Sigma_I_L,x_lim +1*Sigma_I_L])
        
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
            ax.legend()
        
            plt.show()
    return ion_final, ion_final_,starting_pos, loss_history, gradN_3D

# Evaluacion de la energía final

In [ ]:
def test_step(sample_atoms,sample_elements,ion_final):
    #Calculamos la energía al final del entrenamiento y corregimos el desplazamiento del epsilon
    final_energy, final_energy_per_ion= calculate_final_system_energy(
        sample_atoms,  # Tensor con TODAS las posiciones de átomos de la superficie (Zn + O)
        sample_elements,   # Tensor con las etiquetas correspondientes
        ion_final
        )
        
    # Pasa a numpy para reportes
    system_energy = final_energy.numpy()
    final_energy_per_ion = final_energy_per_ion.numpy()
    return system_energy, final_energy_per_ion

In [ ]:
#Definimos una funcion para mostrar los resultados finales de la evaluaion con la posicion optimizada
def show_results(num_atoms, num_ni, system_energy,final_energy_per_ion,starting_pos):
    """Muestra un resumen del resultado energetico de la optimización del sistema
    Args:
        num_atoms: Cantidad de atomos en la superficie adsorbente (int)
        num_ni: cantidad de iones que se intentaron adsorber (int)
        system_energy: energia final total del sistema (float)
        final_energy_per_ion: energia final de cada ion en el sistema (np.array)
        starting_pos: Posicion inicial de los iones a adsorber antes de la optimización (np.array)
    Returns:
        None"""
    
    print(f'Atomos en la superficie: {num_atoms} - Iones a adsorber: {num_ni}')
    print(f'Energía total del sistema:{system_energy:.5f} ev/atom')
    print(f'Energía promedio de adsorcion por ion:{system_energy/num_ni:.5f} ev/atom')
    print(f"Desviación estándar de energía por ion: {tf.math.reduce_std(final_energy_per_ion):.5f} ev/atom") #Revisar
    distance = ion_final_ - starting_pos
    mean_vector = np.mean(distance, axis=0)  # Promedio en X, Y, Z
    mean_magnitude = np.mean(np.linalg.norm(distance, axis=1))
    print(f'Distancia viajada promedio (vectorial): {mean_vector}')
    print(f'Distancia promedio recorrida (magnitud): {mean_magnitude:.3f} Å')

In [ ]:
# Definimos una funcion para hacer los ultimos plots de resultados
def plot_results(loss_history,elements,positions,starting_pos,ion_final_,num_ni, gradN_3D):
    """Grafica la evolucion del costo y la posicion inicial y final de los iones, aqui tambien deberian ir los plots de
    energía vs # de atomos y timpo vs # de atomos.
    Args:
        loss_history: datos de la evolucion del loss durante el entrenamiento (list)
        elements: etiquetas de tipo de atomo (list)
        positions: coordenadas (x,y,z) de los átomos de la superficie.(np.array)
        starting_pos: Posicion inicial de los iones a adsorber antes de la optimización (np.array)
        ion_final_:posicion final de (los) ion(es), optimizada por el entrenamiento en formato numpy(np.float32)
        num_ni: cantidad de iones que se intentaron adsorber (int)
    Returns:
        None"""
    # --- Gráfica de la evolución de la pérdida ---
    plt.figure(figsize=(10, 5))
    plt.plot(loss_history)
    plt.xlabel("Época")
    plt.ylabel("Pérdida Total (Física + PRI)")
    plt.title("Evolución de la Pérdida durante el Entrenamiento")
    plt.grid(True)
    plt.show()

    # --- Gráfica 3D de posiciones iniciales y finales ---
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Superficie
    colors_map = {"Zn": "grey", "O": "blue"}
    for atom_type in set(elements):
        idx = (elements == atom_type)
        ax.scatter(positions[idx, 0], positions[idx, 1], positions[idx, 2],
                   color=colors_map.get(atom_type, "black"),
                   label=f'Superficie ({atom_type})', s=80, alpha=0.6)
    
    # Posiciones de los iones
    ax.scatter(starting_pos[:, 0], starting_pos[:, 1], starting_pos[:, 2],
               color='orange', label='Iones (Inicial)', s=150, marker='x')
    ax.scatter(ion_final_[:, 0], ion_final_[:, 1], ion_final_[:, 2],
               color='red', label='Iones (Final)', s=150, edgecolors='k')
    
    # Flechas de trayectoria
    for i in range(num_ni):
        ax.quiver(starting_pos[i, 0], starting_pos[i, 1], starting_pos[i, 2],
                  gradN_3D[i, 0], gradN_3D[i, 1], gradN_3D[i, 2],
                  color='green', arrow_length_ratio=0.3, alpha=0.7)
    
    ax.set_xlabel("X (Å)")
    ax.set_ylabel("Y (Å)")
    ax.set_zlabel("Z (Å)")
    ax.set_title("Posiciones Iniciales y Finales de los Iones de Ni")
    ax.legend()
    plt.tight_layout()
    plt.show()

#igual que con otras funciones, quiza ademas de .show, deberia guardar los plots en formato png o html

In [ ]:
# Definimos una funcion que llame todo lo relacionado con la evaluacion final
def test_and_review(sample_atoms,sample_elements,ion_final,num_atoms, num_ni,starting_pos,loss_history,elements,positions,ion_final_, gradN_3D):
    """Llama todas las funciones referentes a la evaluacion de resultados"""

    system_energy, final_energy_per_ion = test_step(sample_atoms,sample_elements,ion_final)
    show_results(num_atoms, num_ni, system_energy,final_energy_per_ion,starting_pos)
    plot_results(loss_history,elements,positions,starting_pos,ion_final_,num_ni,gradN_3D)
    

# Muestra 1. ZnO en forma grafeno

In [ ]:
# Parametros de creación de la muestra

a = 1.863  #Distancia de enlace Zn-O (A)
beta = 30*np.pi/180 #angulo de proyeccion de los lados del hexágono sobre el eje x en radianes
dist_x = 2*a*np.cos(beta) # distancia entre puntos en el eje x
dist_y = a*np.sin(beta) # distancia entre puntos en el eje y
offset_x = dist_x / 2 #offset de las filas impares a partir de la tercera fila
cols = 6# Número de columnas y filas
rows = 6


In [ ]:
#parametros de la simulacion para esta muestra
# Parametros del potencial L-J
kj_mol_to_ev = 0.01034 #Factor de conversion kj/mol a eV/atomo
SigmaO = 3.71 #Radio de vanderwalls del Oxigeno (A)
EpsilonO = 1.736 * kj_mol_to_ev #Fondo del pozo del Oxigeno (eV/atom)
SigmaNi = 2.834 #Radio de vanderwalls del Niquel (A)
EpsilonNi = 0.0628 *kj_mol_to_ev #Fondo del pozo Niquel (ev/atom)
SigmaZn = 4.045 #Radio de vanderwalls del Zinc (A)
EpsilonZn = 0.23 *kj_mol_to_ev #Fondo del pozo Zinc (ev/atom)
SigmaO_Ni = (SigmaO+SigmaNi)/2 #Constante sigma de interacción cruzada O-Ni 
EpsilonO_Ni = np.sqrt(EpsilonO*EpsilonNi) #Constante epsilon de interacción cruzada O-Ni 
SigmaZn_Ni = (SigmaZn+SigmaNi)/2 #Constante sigma de interacción cruzada Zn-Ni
EpsilonZn_Ni= np.sqrt(EpsilonZn*EpsilonNi) #Constante epsilon de interacción cruzada Zn-Ni 

# Parametros potencial coulomb
k_e = 14.3996 #Constante de coulomb en vacio en (eV A/e^2)
q_Ni = 2 #(carga del electron e)
q_O = -0.86 #(carga del electron e)
q_Zn = 0.86 #(carga del electron e)

# Radios de corte de intereacción basados en el potencial L-J
r_cutoffO_Ni = 2.5*SigmaO_Ni #Radio maximo de interacción Ni-O(A)
r_cutoffZn_Ni = 2.5*SigmaZn_Ni #Radio maximo de interacción Ni-Zn(A)
r_cutoffNi_Ni = 2.5*SigmaNi #Radio maximo de interacción Ni-Ni(A)

In [ ]:
# Casteamos a tf los parametros de simulacion
SigmaO_Ni = tf.cast(SigmaO_Ni, tf.float32)
EpsilonO_Ni = tf.cast(EpsilonO_Ni, tf.float32)
SigmaZn_Ni = tf.cast(SigmaZn_Ni, tf.float32)
EpsilonZn_Ni = tf.cast(EpsilonZn_Ni, tf.float32)
SigmaNi_Ni = tf.cast(SigmaNi, tf.float32)
EpsilonNi_Ni = tf.cast(EpsilonNi, tf.float32)
k_e = tf.cast(k_e,tf.float32)
q_Ni = tf.cast(q_Ni,tf.float32)
q_O = tf.cast(q_O,tf.float32)
q_Zn = tf.cast(q_Zn,tf.float32)
r_cutoffO_Ni = tf.cast(r_cutoffO_Ni, tf.float32)
r_cutoffZn_Ni = tf.cast(r_cutoffZn_Ni, tf.float32)
r_cutoffNi_Ni = tf.cast(r_cutoffNi_Ni, tf.float32)

In [ ]:
#Definimos la función que crea la muestra
def muestra_ZnO_grafeno(rows,a,dist_y,offset_x,cols,dist_x):
    """Genera una muestra de ZnO en forma grafeno que sigue los parametros definidos anteriormente y retorna las coordenadas 
    que la forman, junto con el tipo de muestra que es.
    Args: 
        rows: cantidad de filas de atomos en la muestra (int)
        a: parametro de red, en este caso longitud de enlace (float)
        dist_y: distancia entre 2 atomos en el eje y (float)
        offset_x: offset entre atomos en el eje x (float)
        cols: cantidad de columnas de atomos (int)
        dist_x: distancia entre 2 atomos en el eje x (float)
    Returns:
        num_atoms: cantidad de atomos en la muestra (int)
        positions: coordenadas de los atomos de la muestra (np.array)
        elements: etiqueta de elemento de cada atomo (np.array)
        sample_atoms: positions pero en tensorflow (tf.tensor(float32))
        sample_elements: elements pero en tensorflow (tf.tensor(str))
        geometria: variable de ayuda para decir que funcion crea una muestra planar (str)
        """
    # Generar diccionario de coordenadas
    coord_dict = {"Zn": [], "O": []}
    
    for row in range(rows):
        y_zn = row * (a + dist_y)
        y_o = y_zn + a
        x_offset = offset_x if row % 2 == 1 else 0
    
        for col in range(cols):
            x = col * dist_x + x_offset
            coord_dict["Zn"].append((x, y_zn, 0))
            coord_dict["O"].append((x, y_o, 0))
    
    positions = []
    elements = []
    
    for atom_type, coords in coord_dict.items():
        for pos in coords:
            positions.append(pos)
            elements.append(atom_type)
    
    
    
    num_atoms = len(positions)
    positions = np.array(positions)
    elements = np.array(elements)
    #tensorflow data
    sample_atoms = tf.convert_to_tensor(positions,dtype=tf.float32)
    sample_elements = tf.convert_to_tensor(elements, dtype=tf.string)
    
    colors = {"Zn": "blue", "O": "grey"}
    
    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection='3d')
    
    for atom_type in set(elements):
        idx = [i for i, e in enumerate(elements) if e == atom_type]
        pos = positions[idx]
        ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2],
                   color=colors.get(atom_type, "black"),
                   label=atom_type, s=100, edgecolors='k')
    
    ax.set_xlabel("X (Å)")
    ax.set_ylabel("Y (Å)")
    ax.set_zlabel("Z (Å)")
    ax.set_title("Estructura ZnO con Ni - Vista 3D")
    ax.legend()
    plt.tight_layout()
    plt.show()

    
    
    return num_atoms, positions, elements, sample_atoms,sample_elements, geometria="planar"

In [ ]:
#ejecutamos todo para la muestra 1
puntos_por_eje = 50
k = 1
num_ni = 1
num_trials = 5
Studies,Trials = optimization(positions, puntos_por_eje, k, num_ni,num_trials,random_seed=42)
best_study = optimization_results(Studies, Trials)
epochs = 3
ion_final, ion_final_, starting_pos, loss_history, gradN_3D = training_loop(best_study, epochs,  positions, sample_atoms, sample_elements)
test_and_review(sample_atoms,sample_elements,ion_final,num_atoms, num_ni,starting_pos,loss_history,elements,positions,ion_final_,gradN_3D)